In [36]:
import json
from typing import Optional, Text
import pandas as pd
from clickhouse_driver import Client
from pydantic import BaseModel

In [37]:
CHUNK_SIZE = 100

In [38]:
client = Client(host="0.0.0.0")

In [39]:
client.execute("SHOW DATABASES")

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('system',)]

In [40]:
client.execute("CREATE DATABASE IF NOT EXISTS test_db")

[]

In [41]:
client.execute("CREATE TABLE IF NOT EXISTS test_db.test_table (Id UInt64, EmployeeName TEXT, JobTitle TEXT, \
                BasePay Float64, OvertimePay Float64, OtherPay Float64, Benefits String, TotalPay Float64, \
                TotalPayBenefits Float64, Year UInt32, Notes String, Agency String, Status String) Engine=MergeTree() ORDER BY Id PRIMARY KEY Id")

[]

In [42]:
client.execute("SHOW TABLES FROM test_db")

[('test_table',)]

In [43]:
df = pd.read_csv("./data/sf_salaries.csv", delimiter=",", encoding="utf-8", low_memory=False)

In [44]:
class Data(BaseModel):
    Id: int
    EmployeeName: Optional[Text]
    JobTitle: Optional[Text]
    BasePay: Optional[float]
    OvertimePay: Optional[float]
    OtherPay: Optional[float]
    Benefits: Optional[str]
    TotalPay: Optional[float]
    TotalPayBenefits: Optional[float]
    Year: Optional[int]
    Notes: Optional[str]
    Agency: Optional[str]
    Status: Optional[str]

In [45]:
payload = json.loads(df.to_json(orient="records"))

In [46]:
for i in payload[0:CHUNK_SIZE]:
    d = Data(**i)
    print(f"Now loading values: ({d.Id}, {d.EmployeeName}, {d.JobTitle}, {d.BasePay}, {d.OvertimePay}, {d.OtherPay}, {d.Benefits}, {d.TotalPay}, {d.TotalPayBenefits}, {d.Year}, {d.Agency}, {d.Status})")
    client.execute(f"INSERT INTO test_db.test_table (Id, EmployeeName) VALUES ({d.Id}, {d.EmployeeName})")

Now loading values: (1, NATHANIEL FORD, GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY, 167411.18, 0.0, 400184.25, None, 567595.43, 567595.43, 2011, San Francisco, None)


ServerException: Code: 62.
DB::Exception: Cannot parse expression of type String here: NATHANIEL FORD): While executing ValuesBlockInputFormat. Stack trace:

0. DB::Exception::Exception(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, int, bool) @ 0xb6fa67a in /usr/bin/clickhouse
1. DB::ValuesBlockInputFormat::parseExpression(DB::IColumn&, unsigned long) @ 0x171c8762 in /usr/bin/clickhouse
2. DB::ValuesBlockInputFormat::readRow(std::__1::vector<COW<DB::IColumn>::mutable_ptr<DB::IColumn>, std::__1::allocator<COW<DB::IColumn>::mutable_ptr<DB::IColumn> > >&, unsigned long) @ 0x171c653c in /usr/bin/clickhouse
3. DB::ValuesBlockInputFormat::generate() @ 0x171c57b3 in /usr/bin/clickhouse
4. DB::ISource::tryGenerate() @ 0x17031115 in /usr/bin/clickhouse
5. DB::ISource::work() @ 0x17030dba in /usr/bin/clickhouse
6. DB::ExecutionThreadContext::executeTask() @ 0x17051528 in /usr/bin/clickhouse
7. DB::PipelineExecutor::executeStepImpl(unsigned long, st